### Bernoulli Naive-Bayes Model

In [1]:
import numpy as np
import pandas as pd

np.set_printoptions(precision=4, suppress=True)

In [2]:
np.random.seed(0)

data = np.random.rand(200,5)
data = np.round(data, decimals=0)
X_train = data[:150, :-1]
y_train = data[:150, -1]

X_test = data[150:, :-1]
y_test = data[150:, -1]

In [3]:
X_test.shape, y_test.shape

((50, 4), (50,))

Base Model

In [4]:
from sklearn.naive_bayes import BernoulliNB
gnb = BernoulliNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

In [5]:
np.sum(y_pred == y_test) / len(y_test)

0.48

In [6]:
class Naive_Bayes:
    """Implements Bernoulli Naive-Bayes with Laplace Smoothing"""
    def compute_Px1y1(self, X, y):
        return (np.sum(X[y==1], axis=0) + 1) / (np.sum(y==1) + self.no_feat)
    
    def compute_Px1y0(self, X, y):
        return (np.sum(X[y==0], axis=0) + 1) / (np.sum(y==0) + self.no_feat)
    
    def compute_Px0y0(self, X, y):
        return 1 - self.compute_Px1y0(X, y)
    
    def compute_Px0y1(self, X, y):
        return 1 - self.compute_Px1y1(X, y)
    
    def compute_Py1(self, y):
        n = len(y)
        return (1/n) * np.sum(y==1)
    
    def compute_Py0(self, y):
        return 1 - self.compute_Py1(y)
    
    def fit(self, X, y):
        self.no_feat = X.shape[1]
        # compute Px=i|y=i
        self.PhiX1y1 = self.compute_Px1y1(X, y)
        self.PhiX1y0 = self.compute_Px1y0(X, y)
        self.PhiX0y0 = self.compute_Px0y0(X, y)
        self.PhiX0y1 = self.compute_Px0y1(X, y)
                      
        self.Phiy1 = self.compute_Py1(y)
        self.Phiy0 = self.compute_Py0(y)

    def predict_proba(self, X):
        Pxy0_i = np.zeros(X.shape)
        Pxy1_i = np.zeros(X.shape)
        n, d = X.shape
        
        assert(d == self.no_feat)
        
        for col in range(d):
            np.put(Pxy1_i[:, col], np.where(X[:, col] == 1), self.PhiX1y1[col])
            np.put(Pxy0_i[:, col], np.where(X[:, col] == 1), self.PhiX1y0[col])
            np.put(Pxy1_i[:, col], np.where(X[:, col] == 0), self.PhiX0y1[col])
            np.put(Pxy0_i[:, col], np.where(X[:, col] == 0), self.PhiX0y0[col])
        
        # Compute Px|y
        Pxy1 = np.multiply.reduce(Pxy1_i, axis=1)
        Pxy0 = np.multiply.reduce(Pxy0_i, axis=1)
        
        # Compute Py|x = Px|y * Py
        Py0 = Pxy0 * self.Phiy0
        Py0 = Py0.reshape(-1, 1)
        
        Py1 = Pxy1 * self.Phiy1
        Py1 = Py1.reshape(-1, 1)
                        
        return np.concatenate((Py0, Py1), axis=1)
    
    def predict(self, X):
        proba = self.predict_proba(X)
        return np.argmax(proba, axis=1)
    
    def generate_new_feature(self):
        "Source: https://www.quora.com/How-do-I-generate-data-using-a-Naive-Bayes-model"
                   
        return 
    

In [7]:
nb = Naive_Bayes()

In [8]:
nb.fit(X_train, y_train)

In [9]:
predictions = nb.predict(X_test)

In [10]:
predictions

array([1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1])

In [11]:
np.sum(predictions == y_test) / len(y_test)

0.48

In [12]:
nb.predict_proba(X_test)[0:5]

array([[0.0343, 0.036 ],
       [0.0265, 0.025 ],
       [0.0265, 0.025 ],
       [0.0343, 0.0226],
       [0.0265, 0.025 ]])